In [12]:
import tkinter as tk
from tkinter import messagebox
from PIL import Image, ImageTk
import numpy as np
import joblib
import os
from catboost import CatBoostRegressor

# Initialize the main window
root = tk.Tk()
root.title("Predicting Discharge Cofficient of PKW")

# Create the main canvas
canvas = tk.Canvas(root, width=780, height=420, background='#FFFFFF')
canvas.pack()

# Load the image from the desktop and create an image on the canvas using PIL
image_path = r"C:\Users\asus1\Desktop\image.png"  # Use a raw string for the file path

# Load the image from the specified path and create an image on the canvas using PIL
try:
    image = Image.open(image_path)
    root.image_image = ImageTk.PhotoImage(image)  # Store the image
    canvas.create_image(250, 40, image=root.image_image, anchor="n")
except Exception as e:
    messagebox.showerror("Error", f"Failed to load image: {e}")
    

# Function to create labels on the canvas
def create_label(text, font, fg, bg, x, y):
    label = tk.Label(root, text=text, font=font, fg=fg, bg=bg)
    canvas.create_window(x, y, anchor="w", window=label)
    return label

# Check if the CatBoost model file exists
model_path_catb = os.path.expanduser("~/Desktop/CTB.joblib")
if not os.path.exists(model_path_catb):
    messagebox.showerror("Error", "CatBoost model file not found.")
    raise SystemExit  # Exit if the model file is not found

# Load the CatBoost model
model_catb = joblib.load(model_path_catb)

# Define the GUI elements
label_inputdefinetitle2 = create_label('Predicting Discharge Coefficient of Piano Key Weir (PKW)', ('Palatino Linotype', 18, 'bold', 'underline'), '#0000FF', '#FFFF00', 75,20)
label_inputdefinetitle = create_label('Definition of Parameters', ('Palatino Linotype', 14, 'bold', 'italic'), '#000000', '#FFFFFF', 50,300)

param_labels = [
    'X1: PKW key widths ratio (W*)',
    'X2: PKW height ratio (P*)',
    'X3: Total upstream head ratio (H*)',
    'Y: Discharge coefficient (Cd)',
]
for i, text in enumerate(param_labels, start=1):
    create_label(text, ('Georgia', 14, 'italic'), '#00008B', '#FFFFFF', 50, 270 + i*30)

label_inputs = create_label('Inputs', ('Palatino Linotype', 17, 'bold', 'italic'), '#000000', '#FFFFFF', 580, 80)

entry_fields = ['X1', 'X2', 'X3']
entries = {}
for i, field in enumerate(entry_fields, start=1):
    label = create_label(f'{field}: ', ('Georgia', 16), '#FF0000', '#FFFFFF', 530, 100 + i*50)
    entry = tk.Entry(root)
    canvas.create_window(650, 100 + i*50, window=entry)
    entries[field] = entry

label_output = create_label('Output', ('Palatino Linotype', 17, 'bold', 'italic'), '#000000', '#FFFFFF', 580, 300)
label_result = create_label('Cd', ('Georgia', 18, 'italic'), '#FF0000', '#FFFFFF', 600, 350)

# Predict button command function
def predict():
    # Collect input data
    input_values = []
    for field in entry_fields:
        try:
            value = float(entries[field].get())
            if value <= 0:
                raise ValueError("Value must be positive.")
            input_values.append(value)
        except ValueError as e:
            messagebox.showerror("Error", f"Invalid input for {field}. Please enter a positive numeric value.")
            return

    input_data = np.array([input_values])
    try:
        prediction = model_catb.predict(input_data)
        label_result['text'] = f'Cd = {prediction[0]:.4f}'
        label_result.config(background='yellow', relief='solid', borderwidth=1)
    except Exception as e:
        messagebox.showerror("Error", f"An error occurred during prediction: {e}")

# Predict button
predict_button = tk.Button(root, text='Calculate', font=('Georgia', 14), command=predict, fg='#FFFFFF', bg='green')
canvas.create_window(720, 300, window=predict_button)

# Run the main loop
root.mainloop()